In [ ]:
# Header starts here.
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
from copy import deepcopy
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        1,              #  1
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    objc = deepcopy(obj)
    try:
        tmp = DX(str(float(objc)))
        objc = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(objc)):
            tmp = DX(str(float(objc[i])))
            objc[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return objc

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

half = S(1)/2

# Header ends here.
#
# https://colab.research.google.com/github/kassbohm/wb-snippets/blob/master/ipynb/HTM_03/Selbst/3.3_cc.ipynb

from mpmath import radians, degrees, pi

# SI Units:
l = 1*meter/10
a = 1*meter/10
b = 1*meter/100
w1 = 2*pi /s

dp2, dp3 = var("delta_2, delta_3")

pprint("\nphi1 / deg:")
p1_deg = 30
# p1_deg = 60
p1 = p1_deg *pi/180
pprint(p1_deg)

c1, s1 = cos(p1), sin(p1)

p2, p3 = var("phi2, phi3",positive=True)

c2, s2 = cos(p2), sin(p2)
c3, s3 = cos(p3), sin(p3)

rB = l/2 * Matrix([c1, s1, 0])
rD = Matrix([a, -b, 0])

dBC = l * Matrix([c2, s2, 0])
dDC = l * Matrix([c3, s3, 0])

dev = rB + dBC - (rD + dDC)
xdev = dev[0] /meter
ydev = dev[1] /meter

J11, J12 = diff(xdev, p2), diff(xdev, p3)
J21, J22 = diff(ydev, p2), diff(ydev, p3)

# Jacobian:
J = Matrix([[J11, J12],[J21, J22]])

pprint("\n--- Numerical Solution using SymPy's nsolve ---")
# Initial Values:
# pprint("\nInitial (phi2, phi3) / deg:")
# pprint(Matrix([0, 90]))

p2i, p3i = radians(0) , radians(90)
sol = nsolve( [xdev, ydev], [p2, p3], [p2i, p3i], dict=True )
sol = sol[0]
p2_sol, p3_sol =  sol[p2], sol[p3]

sol =  Matrix([sol[p2], sol[p3]])

pprint("\n(phi2, phi3) / deg (rounded to 0.01):")
tmp = degrees(sol)
pprint(iso_round(tmp,0.01))


# pprint("\n--- Step-by-Step Newton-Solution ---")
# pprint("\n1: Jacobian:")
# pprint(J)
#
# # n = 1 Iteration:
# pprint("\n2: Init:")
# p = Matrix([p2i, p3i])
# pprint(iso_round(p, 0.0001))
#
# F = Matrix([xdev, ydev])
#
# sub_list = [
#     (p2, p2i),
#     (p3, p3i),
#     ]
#
# pprint("\n3: Jacobian for (phi2, phi3):")
# Jn = J.subs(sub_list)
# # pprint(Jn)
# pprint(iso_round(Jn, 0.0001))
#
# pprint("\n4: Deviation Phi for (phi2, phi3):")
# Fn = F.subs(sub_list)
# pprint(iso_round(Fn, 0.0001))
#
# # Newton-Step:
# pprint("\n5: New (phi2, phi3):")
# dp = Matrix([dp2, dp3])
# eq = Eq(Jn*dp, -Fn)
# sol = solve(eq,[dp2, dp3], dict=True)
# sol = sol[0]
# dp = Matrix([sol[dp2], sol[dp3]])
# p += dp
#
# pprint("\nIn rad (radians):")
# pprint(iso_round(p, 0.0001))
#
# pprint("\nIn deg (degrees):")
# pprint(iso_round(degrees(p), 0.1))
#
# exit()


# Further results:
p2, p3 =  p2_sol, p3_sol
c2, s2 = cos(p2), sin(p2)
c3, s3 = cos(p3), sin(p3)

dBC = l * Matrix([c2, s2,0])
rC = rB + dBC

pprint("\nrC / mm (rounded to 0.1):")
tmp = rC
tmp /= mm
tmp = iso_round(tmp, 0.1)
pprint(tmp)

pprint("\nvB / (mm/s) (rounded to 0.1):")
O1 = Matrix([0,0,w1])
vB = O1.cross(rB)
tmp = vB
tmp /= mm/s
tmp = iso_round(tmp, 0.1)
pprint(tmp)

# phi1 / deg:
# 60
#
# --- Numerical Solution using SymPy's nsolve ---
#
# (phi2, phi3) / deg (rounded to 0.01):
# ⎡27.21⎤
# ⎢     ⎥
# ⎣81.99⎦
#
# rC / mm (rounded to 0.1):
# ⎡113.9⎤
# ⎢     ⎥
# ⎢89.0 ⎥
# ⎢     ⎥
# ⎣ 0.0 ⎦
#
# vB / (mm/s) (rounded to 0.1):
# ⎡-272.1⎤
# ⎢      ⎥
# ⎢157.1 ⎥
# ⎢      ⎥
# ⎣ 0.0  ⎦
